In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Emotion Classification (FER-2013)

Bu projede 48x48 piksel gri tonlamalı görüntüler kullanılarak, her yüzü yüz ifadesinde gösterilen duyguya göre yedi kategoriden birine (0=Öfkeli, 1=İğrenme, 2=Korku, 3=Mutlu, 4=Üzgün, 5=Şaşkın, 6=Nötr) göre sınıflandıran model geliştirilmektedir. Eğitim seti 28.709 örnekten ve genel test seti 3.589 örnekten oluşur.

Bu projede kullanılan veriler [buradan](https://www.kaggle.com/datasets/msambare/fer2013) temin edilmiştir.

<img src="https://storage.googleapis.com/kaggle-datasets-images/786787/1351797/89e6e907cb903b4f523bbd613e46b8a8/dataset-cover.png?t=2020-07-19-18-42-52">

In [ ]:
import cv2
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
img_path="/kaggle/input/fer2013/train/"
labels = [item for item in os.listdir(img_path) if os.path.isdir(os.path.join(img_path, item))]
labels

In [ ]:
def set_df(dir):
    img_list=[]
    label_list=[]
    for label in labels:
        for img_file in os.listdir(dir+label):
            img_list.append(dir+label+"/"+img_file)
            label_list.append(label)
    df=pd.DataFrame({"img":img_list, "label":label_list})
    return df

In [ ]:
train=set_df("/kaggle/input/fer2013/train/")
test=set_df("/kaggle/input/fer2013/test/")

In [ ]:
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)
train.head(10)

In [ ]:
sns.countplot(x=train["label"])

In [ ]:
sns.countplot(x=test["label"])

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train["label"])
train["encode_label"] = label_encoder.transform(train["label"])
test["encode_label"] = label_encoder.transform(test["label"]) 
 
train.head()

In [ ]:
classes=label_encoder.classes_
classes

In [ ]:
selected_images = train.sample(24)
selected_images = selected_images.reset_index(drop=True)

fig, axes = plt.subplots(3, 8, figsize=(12, 7))

for i, ax in enumerate(axes.flat):
    image=cv2.imread(str(selected_images["img"][i]))
    ax.imshow(image)
    ax.axis("off")
    ax.text(0.5, -0.2, classes[selected_images["encode_label"][i]], transform=ax.transAxes, ha='center')
plt.tight_layout()
plt.show()

In [ ]:
size=48
x_train=[]
for imge in train["img"]:
    img=cv2.imread(str(imge))
    img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img=cv2.resize(img, (size,size))
    img=img/255.0 
    x_train.append(img)

In [ ]:
x_test=[]
for imge in test["img"]:
    img=cv2.imread(str(imge))
    img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img=cv2.resize(img, (size,size))
    img=img/255.0 
    x_test.append(img)

## Modelleme

In [ ]:
x_train=np.array(x_train)
x_test=np.array(x_test)

y_train=train['encode_label']
y_test=test['encode_label']

In [ ]:
print(y_train[0])
plt.imshow(x_train[0])

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout,BatchNormalization, Reshape
from sklearn.metrics import f1_score, confusion_matrix, classification_report

In [ ]:
model=Sequential()
model.add(Input(shape=(size,size,1)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=.5))
model.add(Dense(7, activation='softmax'))
#categorical_crossentropy
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=32, epochs=50, verbose=1)

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
loss, accuracy

In [ ]:
plt.plot(history.history["accuracy"], label="Accuracy")
plt.plot(history.history["val_accuracy"], label="Val Accuracy")
plt.legend()

In [ ]:
pred = model.predict(x_test)

In [ ]:
confusion_matrix(y_test, np.argmax(pred, axis=1))

In [ ]:
sns.heatmap(confusion_matrix(y_test, np.argmax(pred, axis=1)), annot=True);

In [ ]:
print(classification_report(y_test, np.argmax(pred, axis=1)))

In [ ]:
def tahmin(img):
    img=cv2.resize(image, (size,size))
    img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img=img/255.0 
    img=np.array(img)
    img=np.expand_dims(img, axis=0)
    thmn=model.predict(img)
    thmn=np.argmax(thmn)
    return thmn

selected_images = test.sample(24)
selected_images = selected_images.reset_index(drop=True)

fig, axes = plt.subplots(3, 8, figsize=(12, 7))

for i, ax in enumerate(axes.flat):
    image=cv2.imread(str(selected_images["img"][i]))
    ax.imshow(image)
    ax.axis("off")
    pred=tahmin(image)
    ax.text(0.5, -0.2, "True:" + classes[selected_images["encode_label"][i]], transform=ax.transAxes, ha='center')
    ax.text(0.5, -0.4, "Predicted:" + classes[pred], transform=ax.transAxes, ha='center')
plt.tight_layout()
plt.show()

In [ ]:
model.save("emotion_model.keras")

## Sonuç:
7 farklı duygu çeşitini öğrenip, tahmin eden bir model geliştirildi. Modelin Eğitim başarısı yaklaşık %98 iken Doğrulama başarısı yaklaşık %57 olmuştur. Modeldeki test verileri içinden rastgele seçilen 24 resim yeniden tahmin edilmiş ve 13 tanesinin doğru tahmin edildiği görülmüştür. Genellikle happy, neutral ve angry türlerinde daha fazla doğru tahmin ettiği düşünülmektedir.